Some parts of `pygsti` are works-in-progress. Here, we investigate how to do the task of "model selection" within GST, essentially answering the question "Can we do a better job of modeling the experiment by changing the assumptions within GST?".

## Testing variable-gateset-dimension GST with model selection

###     Setup

In [1]:
import pygsti

In [2]:
#Load gateset and some string lists
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
specs = pygsti.construction.build_spam_specs(fiducialList)
expList = [1,2,4]


In [3]:
#Create some testing gate string lists
lgstList = pygsti.construction.list_lgst_gatestrings(specs, gs_target.gates.keys())
lsgstLists = [ lgstList[:] ]
for exp in expList:
    gsList = pygsti.construction.create_gatestring_list(
                "f0+germ*exp+f1", f0=fiducialList, f1=fiducialList,
                germ=germList, exp=exp, order=['germ','f0','f1'])
    lsgstLists.append( lsgstLists[-1] +  gsList )
    
dsList = pygsti.remove_duplicates( lsgstLists[-1] )

In [4]:
#Test on fake data by depolarizing target set, increasing its dimension,
# and adding leakage to the gates into the new dimension.

gs_dataGen4 = gs_target.depolarize(gate_noise=0.1)
gs_dataGen5 = gs_dataGen4.increase_dimension(5)
leakGate = pygsti.construction.build_gate( [2,1],[('Q0',),('L0',)] , "LX(pi/4.0,0,2)","gm") # X(pi,Q0)*LX(pi,0,2)

gs_dataGen5['Gx'] = pygsti.objects.compose( gs_dataGen5['Gx'], leakGate)
gs_dataGen5['Gy'] = pygsti.objects.compose( gs_dataGen5['Gy'], leakGate)
print(gs_dataGen5.gates.keys())

#Some debugging...
#NOTE: with LX(pi,0,2) above, dim 5 test will choose a dimension 3 gateset, which may be sensible
#       looking at the gate matrices in this case... but maybe LX(pi,...) is faulty?
#print gs_dataGen4
#print gs_dataGen5

#Jmx = GST.JOps.jamiolkowski_iso(gs_dataGen4['Gx'])
#Jmx = GST.JOps.jamiolkowski_iso(gs_dataGen5['Gx'],dimOrStateSpaceDims=[2,1])
#print "J = \n",Jmx
#print "evals = ",eigvals(Jmx)

dsFake4 = pygsti.construction.generate_fake_data(gs_dataGen4, dsList, nSamples=1000000, sampleError="binomial", seed=1234)
dsFake5 = pygsti.construction.generate_fake_data(gs_dataGen5, dsList, nSamples=1000000, sampleError="binomial", seed=1234)

odict_keys(['Gi', 'Gx', 'Gy'])


In [5]:
print("Number of gates                        = ",len(gs_target.gates.keys()))
print("Number of fiducials                    =",len(fiducialList))
print("Maximum length for a gate string in ds =",max(map(len,dsList)))
print("Number of LGST strings                 = ",len(lgstList))
print("Number of LSGST strings                = ",map(len,lsgstLists))

Number of gates                        =  3
Number of fiducials                    = 6
Maximum length for a gate string in ds = 30
Number of LGST strings                 =  92
Number of LSGST strings                =  <map object at 0x7fccfc804b00>


### Test using dimension-4 fake data

In [6]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
# NOTE: with nSamples less than 1M (100K, 10K, 1K) this routine will choose a higher-than-4 dimensional gateset
ds = dsFake4
gs_lgst4 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=3)
gs_lgst6 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=6, verbosity=3)

#Print chi^2 of 4-dim and 6-dim estimates
chiSq4 = pygsti.chi2(ds, gs_lgst4, lgstList, minProbClipForWeighting=1e-4)
chiSq6 = pygsti.chi2(ds, gs_lgst6, lgstList, minProbClipForWeighting=1e-4)
print("LGST dim=4 chiSq = ", chiSq4)
print("LGST dim=6 chiSq = ", chiSq6)

# Least squares GST with model selection
gs_lsgst = pygsti.do_iterative_mc2gst_with_model_selection(ds, gs_lgst4, 1, lsgstLists, verbosity=2,
                                                           minProbClipForWeighting=1e-3, probClipInterval=(-1e5,1e5))

    
--- LGST ---
    ('LGST: Singular values of I_tilde (truncating to first 4 of 6) = \n', array([  3.00262982e+00,   8.25857216e-01,   6.71539150e-01,
         6.64532662e-01,   8.38614352e-04,   2.92355240e-04]))
      LGST DEBUG: Gi before trunc to first 4 row and cols = 

   3.0025   0.0038  -0.0003  -0.0078   0.0003  -0.0003
  -0.0032   0.7431  -0.0009  -0.0006  -0.0009  -0.0008
   0.0009   0.0006   0.6038   0.0006   0.0003   0.0001
  -0.0077   0.0001  -0.0009   0.5986   0.0002  -0.0004
        0   0.0008   0.0002   0.0008   0.0004  -0.0004
  -0.0004  -0.0017        0   0.0011  -0.0004  -0.0008

      LGST DEBUG: Gx before trunc to first 4 row and cols = 

   3.0050   0.0913  -0.0109  -0.0190  -0.0002  -0.0008
  -0.0905   0.0016  -0.4699  -0.4744        0        0
  -0.0107   0.4712   0.3068  -0.2994  -0.0006   0.0007
  -0.0188   0.4731  -0.3016   0.2924        0  -0.0003
   0.0004        0  -0.0010        0        0  -0.0010
  -0.0002        0   0.0008  -0.0003        0        

In [7]:
print(gs_lsgst)

rho0 =    0.7072  -0.0182   0.0176   0.7504


E0 =    0.6861   0.0059  -0.0023  -0.6463


Gi = 
   1.0000        0        0        0
  -0.0021   0.9001   0.0001   0.0003
   0.0032        0   0.8998   0.0002
  -0.0032  -0.0001   0.0002   0.9000


Gx = 
   1.0000        0        0        0
  -0.0019   0.8997  -0.0041   0.0042
  -0.0011  -0.0120  -0.0084  -0.9993
  -0.0576   0.0273   0.8107   0.0086


Gy = 
   1.0000        0        0        0
   0.0122  -0.0055   0.0037   1.0086
   0.0032   0.0264   0.8999  -0.0180
  -0.0491  -0.8032  -0.0031   0.0057





### Test using dimension-5 fake data

In [8]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
ds = dsFake5
gs_lgst4 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=3)
gs_lgst6 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=6, verbosity=3)

#Print chi^2 of 4-dim and 6-dim estimates
chiSq4 = pygsti.chi2(ds, gs_lgst4, lgstList, minProbClipForWeighting=1e-2)
chiSq6 = pygsti.chi2(ds, gs_lgst6, lgstList, minProbClipForWeighting=1e-2)
print("LGST dim=4 chiSq = ", chiSq4)
print("LGST dim=6 chiSq = ", chiSq6)

# Least squares GST with model selection
gs_lsgst = pygsti.do_iterative_mc2gst_with_model_selection(ds, gs_lgst4, 1, lsgstLists, verbosity=2, minProbClipForWeighting=1e-3, probClipInterval=(-1e5,1e5), useFreqWeightedChiSq=False, regularizeFactor=1.0, check=False, check_jacobian=False)

    
--- LGST ---
    ('LGST: Singular values of I_tilde (truncating to first 4 of 6) = \n', array([  3.56525557e+00,   7.55465957e-01,   4.96239591e-01,
         4.87203220e-01,   6.91003018e-03,   3.21987001e-04]))
      LGST DEBUG: Gi before trunc to first 4 row and cols = 

   3.5655  -0.0003  -0.0003   0.0054   0.0003        0
  -0.0079   0.6814  -0.0012   0.0003        0   0.0009
   0.0004   0.0007   0.4459  -0.0007        0  -0.0002
   0.0067   0.0009   0.0008   0.4387  -0.0008   0.0001
   0.0002  -0.0016  -0.0004   0.0014   0.0063  -0.0005
  -0.0002  -0.0017        0  -0.0004   0.0004   0.0005

      LGST DEBUG: Gx before trunc to first 4 row and cols = 

   3.6664   0.1371  -0.0208   0.0234  -0.0100   0.0012
  -0.1340  -0.0015  -0.4154   0.2670  -0.0007        0
  -0.0194   0.4102   0.1353   0.1988   0.0105  -0.0004
   0.0177  -0.2737   0.1966   0.2627  -0.0077   0.0007
   0.0108  -0.0007  -0.0108   0.0073   0.0047  -0.0006
  -0.0001        0   0.0007   0.0007  -0.0003        

In [9]:
print(gs_lsgst)

rho0 =    0.7072   0.0847  -0.0849   0.7981   0.0066


E0 =    0.7638   0.0100  -0.0244  -0.6801  -0.0477


Gi = 
   1.0000        0        0        0        0
   0.0073   0.9001  -0.0003        0  -0.0059
  -0.0097  -0.0006   0.9003   0.0003   0.0053
   0.0068  -0.0014   0.0022   0.9016   0.0258
        0  -0.0044   0.0071   0.0055   0.9970


Gx = 
   1.0000        0   0.0001  -0.0002        0
   0.0288   0.8312   0.0441  -0.0119   0.0019
   0.0370  -0.0017  -0.0217  -0.9983   0.3937
   0.0748   0.0002   0.7026   0.0021   0.1630
  -0.2079   0.0027   0.0220  -0.1014   0.8009


Gy = 
   1.0000  -0.0001        0  -0.0002        0
  -0.0579  -0.0284  -0.0098   0.9979  -0.3947
  -0.0306   0.0168   0.8317   0.0138  -0.0057
   0.0592  -0.6985  -0.0232  -0.0037   0.1661
  -0.2059   0.0002  -0.0054  -0.1325   0.8130



